In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35577788
paper_name = 'sing_pon_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [14]:
original_data = pd.read_csv('raw_data/resistant.txt', header=None, names=['genes'])

In [15]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 18 x 1


In [16]:
original_data.head()

,genes
0,SKI8
1,YHR022C
2,IDH1
3,SNO4
4,MSH6


In [17]:
original_data['genes'] = original_data['genes'].astype(str)

In [18]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [19]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [20]:
original_data.loc[original_data['orf']=='ROQ1','orf'] = 'YJL144W'

In [21]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             genes    orf
index_input              
13           CCW22  CCW22


In [22]:
original_data = original_data.loc[t,]

In [23]:
original_data['data'] = 1

In [24]:
original_data.set_index('orf', inplace=True)

In [25]:
original_data = original_data[['data']].copy()

In [26]:
original_data = original_data.groupby(original_data.index).mean()

In [27]:
original_data.shape

(17, 1)

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [22229]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,22229
data_type,value
orf,
YDR097C,1
YDR126W,1
YDR127W,1
YDR137W,1
YDR403W,1


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22229
,data_type,value
gene_id,orf,
2224,YDR097C,1
2253,YDR126W,1
2254,YDR127W,1
2263,YDR137W,1
2530,YDR403W,1


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22229           
data_type       value     valuez
gene_id orf                     
2224    YDR097C     1  16.367112
2253    YDR126W     1  16.367112
2254    YDR127W     1  16.367112
2263    YDR137W     1  16.367112
2530    YDR403W     1  16.367112

# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')